In [ ]:
!pip install llama-index
!pip install llama-index-llms-huggingface
!pip install llama-index-embeddings-huggingface

In [ ]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Get the access token from the environment variable
access_token = os.getenv("HUGGINGFACE_ACCESS_TOKEN")
assert access_token


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
    "meta-llama/Meta-Llama-3-8B-Instruct", token=access_token
)



stopping_ids = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>"),
]

In [ ]:
# generate_kwargs parameters are taken from https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct

from llama_index.llms.huggingface import HuggingFaceLLM

llm = HuggingFaceLLM(
    model_name="meta-llama/Meta-Llama-3-8B-Instruct",
    generate_kwargs={
        "do_sample": True,
        "temperature": 0.6,
        "top_p": 0.9,
    },

    tokenizer_name="meta-llama/Meta-Llama-3-8B-Instruct",
    stopping_ids=stopping_ids,
    model_kwargs={"token": access_token}
)

In [ ]:
response = llm.complete("Who is Paul Graham?")

print(response)

# RAG Pipeline

In [ ]:
import requests

url = "https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt"
output_file = "paul_graham_essay.txt"

response = requests.get(url)
with open(output_file, 'wb') as f:
    f.write(response.content)



In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["paul_graham_essay.txt"]
).load_data()

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

In [ ]:
from llama_index.core import Settings

# bge embedding model
Settings.embed_model = embed_model

# Llama-3-8B-Instruct model
Settings.llm = llm

In [ ]:
index = VectorStoreIndex.from_documents(
    documents,
)

In [ ]:
query_engine = index.as_query_engine(similarity_top_k=3)

In [ ]:
response = query_engine.query("What did paul graham do growing up?")

In [ ]:
print(response)